In [ ]:
!pip install pandas
!pip install numpy
!pip install scikit-learn

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import pandas as pd
import numpy as np

data = pd.read_csv('./weather.csv')
print(data.head())

    outlook temperature humidity  windy play
0  overcast         hot     high  False  yes
1  overcast        cool   normal   True  yes
2  overcast        mild     high   True  yes
3  overcast         hot   normal  False  yes
4     rainy        mild     high  False  yes


In [ ]:
data.head()

,outlook,temperature,humidity,windy,play
0,overcast,hot,high,False,yes
1,overcast,cool,normal,True,yes
2,overcast,mild,high,True,yes
3,overcast,hot,normal,False,yes
4,rainy,mild,high,False,yes


In [ ]:
from sklearn.naive_bayes import GaussianNB
X = np.asarray(data)

In [ ]:
print(X[:5])

[[5 -1 0 0 1]
 [5 1 1 1 1]
 [5 0 0 1 1]
 [5 -1 1 0 1]
 [0 0 0 0 1]]


In [ ]:
def mapper(row):
  d = {"overcast": -1, "rainy" : 0 , "sunny" : 1, "hot":-1,"cool":1,"mild":0,'high':0,"normal":1,
       True : 1, False: 0, "yes": 1, "no":0}
  
  for i in range(len(row)):
    row[i] = d[row[i]]
  return row

for i in range(len(X)):
  X[i] = mapper(X[i])
print(X[:5])

[[-1 -1 0 0 1]
 [-1 1 1 1 1]
 [-1 0 0 1 1]
 [-1 -1 1 0 1]
 [0 0 0 0 1]]


In [28]:
from functools import reduce

import pandas as pd
import pprint

class Classifier():
    data = None
    class_attr = None
    priori = {}
    cp = {}
    hypothesis = None


    def __init__(self,filename=None, class_attr=None ):
        self.data = pd.read_csv(filename, sep=',', header =(0))
        self.class_attr = class_attr
    def calculate_priori(self):
        class_values = list(set(self.data[self.class_attr]))
        class_data =  list(self.data[self.class_attr])
        for i in class_values:
            self.priori[i]  = class_data.count(i)/float(len(class_data))
        print ("Priori Values: ", self.priori)

    def get_cp(self, attr, attr_type, class_value):
        data_attr = list(self.data[attr])
        class_data = list(self.data[self.class_attr])
        total =1
        for i in range(0, len(data_attr)):
            if class_data[i] == class_value and data_attr[i] == attr_type:
                total+=1
        return total/float(class_data.count(class_value))

    def calculate_conditional_probabilities(self, hypothesis):
        for i in self.priori:
            self.cp[i] = {}
            for j in hypothesis:
                self.cp[i].update({ hypothesis[j]: self.get_cp(j, hypothesis[j], i)})
        print ("\nCalculated Conditional Probabilities: \n")
        pprint.pprint(self.cp)

    def classify(self):
        print ("Result: ")
        for i in self.cp:
            print (i, " ==> ", reduce(lambda x, y: x*y, self.cp[i].values())*self.priori[i])

if __name__ == "__main__":
    c = Classifier(filename="./weather.csv", class_attr="play" )
    c.calculate_priori()
    c.hypothesis = {"outlook":'rainy', "temperature":"mild", "humidity":'normal' , "windy":'t'}

    c.calculate_conditional_probabilities(c.hypothesis)
    c.classify()


Priori Values:  {'no': 0.35714285714285715, 'yes': 0.6428571428571429}

Calculated Conditional Probabilities: 

{'no': {'mild': 0.6, 'normal': 0.4, 'rainy': 0.6, 't': 0.2},
 'yes': {'mild': 0.5555555555555556,
         'normal': 0.7777777777777778,
         'rainy': 0.4444444444444444,
         't': 0.1111111111111111}}
Result: 
no  ==>  0.010285714285714285
yes  ==>  0.013717421124828532
